In [1]:
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
from skimage import feature

from scipy.misc import imsave
from scipy import ndimage as ndi
from scipy.spatial import distance

import scipy.ndimage as ndimage
import scipy.ndimage.morphology as morphology

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import colors

import pandas as pd
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook

from PIL import Image
import os
import warnings

In [2]:
tqdm_notebook.pandas()

In [3]:
warnings.filterwarnings('ignore')

In [4]:
TRAIN_DATA_DIR = '../skoltech-landmarks/landmarks-clean/data_train'
MASK_DIR = 'masks'
if not os.path.exists(MASK_DIR):
    os.mkdir(MASK_DIR)

In [5]:
N_SEGMENTS = 100

In [6]:
df_train = pd.read_csv('../skoltech-landmarks/landmarks-clean/labels_train.csv', sep=';')

In [7]:
def check_image(im, imdir):
    impath = os.path.join(imdir, str(im))
    if os.path.exists(impath):
        im = Image.open(impath)
        w, h = im.size
        if w > 100 and h > 100:
            return True
    return False

In [8]:
df_train.columns = ['image', 'label', 'r1', 'r2', 'r3', 'r4']
df_train = df_train[df_train['image'].apply(lambda x: check_image(x, TRAIN_DATA_DIR))]
groups = df_train.groupby('label')
df_train = groups.filter(lambda x: len(x) > 1)

In [9]:
df_train['rect'] = df_train.apply(lambda x: [x['r1'], x['r2'], x['r3'], x['r4']], axis = 1)

In [10]:
df_train = df_train.drop(['r1', 'r2', 'r3', 'r4'], axis = 1)

In [11]:
def get_positive_segments(segments, rect):
    x1, x2, y1, y2 = rect
    pos_segments = []
    for i, val in enumerate(np.unique(segments)):
        x, y = np.where(segments == val)
        assert len(x) == len(y)
        total = len(x)
        positive = 0
        for j in range(len(x)):
            x_c = x[j]
            y_c = y[j]
            if x1 < x_c < x2 and y1 < y_c < y2:
                positive += 1
        if total > 0:
            if positive/total >= 0.7:
                pos_segments.append(i)
    return pos_segments

In [12]:
def compute_histogram(img, nbins):
    h_h = np.histogram(img[:,:,0], bins=nbins)[0]
    s_h = np.histogram(img[:,:,1], bins=nbins)[0]
    v_h = np.histogram(img[:,:,2], bins=nbins)[0]
    h_h = h_h / np.linalg.norm(h_h)
    s_h = s_h / np.linalg.norm(s_h)
    v_h = v_h / np.linalg.norm(v_h)
    return h_h, s_h, v_h

In [13]:
def compute_sp_histogram(x, y, image, nbins):
    h = []
    s = []
    v = []
    for j in range(len(x)):
        x_c = x[j]
        y_c = y[j]
        h.append(image[x_c, y_c, 0])
        s.append(image[x_c, y_c, 1])
        v.append(image[x_c, y_c, 2])
    h_h = np.histogram(h, bins=nbins)[0]
    s_h = np.histogram(s, bins=nbins)[0]
    v_h = np.histogram(v, bins=nbins)[0]
    h_h = h_h / np.linalg.norm(h_h)
    s_h = s_h / np.linalg.norm(s_h)
    v_h = v_h / np.linalg.norm(v_h)
    return h_h, s_h, v_h

In [14]:
def post_process_sp(image, segments, bbox):
    dists = {}
    image = colors.rgb_to_hsv(image)
    image[np.isnan(image)] = 0
    nbins = 100
    img = image[bbox[0]:bbox[1]-bbox[0], bbox[2]:bbox[3]-bbox[2]]
    f_h, f_s, f_v = compute_histogram(img, nbins)
    full_hist = f_h + f_s + f_v
    for i, segment in enumerate(np.unique(segments)):
        if segment != -1:
            x, y = np.where(segments == segment)
            s_h, s_s, s_v = compute_sp_histogram(x, y, image, nbins)
            seg_hist = s_h + s_s + s_v
            dist = distance.euclidean(full_hist, seg_hist)
            dists[segment] = dist
            #print('segment # {}, distance = {}'.format(segment, dist))
    treshold = np.mean(list(dists.values()))*1.1
    for item in np.unique(segments):
        if item != -1:
            if dists[item] > treshold:
                segments[np.where(segments == item)] = -1
    return segments

In [15]:
def process_spx(image, rect, n_segments):
    segments = slic(image, n_segments, sigma = 5)
    pos_segm = get_positive_segments(segments, rect)
    segments[~np.isin(segments, pos_segm)] = -1
    segments = post_process_sp(image, segments, rect)
    return segments

In [16]:
def process_mask(sgs):
    #create mask
    sgs[np.where(sgs != -1)] = 1
    sgs[np.where(sgs == -1)] = 0
    #fill gaps
    sgs = ndimage.binary_fill_holes(sgs)
    #cut small components
    label_im, nb_labels = ndimage.label(sgs)
    #unite and thrive
    sizes = ndimage.sum(sgs, label_im, range(nb_labels + 1))
    largest_label = np.argmax(sizes)
    label_im[np.where(label_im != largest_label)] = 0
    label_im[np.where(label_im == largest_label)] = 1
    return label_im

In [17]:
def process_image(imname, imdir, rect, n_segments):
    try:
        im = img_as_float(io.imread(os.path.join(imdir, str(imname))))
        im[np.isnan(im)] = 0
        sgs = process_spx(im, rect, n_segments)
        sgs = process_mask(sgs)
        imsave(os.path.join(MASK_DIR, str(imname)+'.jpg'), sgs)
    except:
        pass

In [18]:
df_train.progress_apply(lambda x: process_image(x['image'],
                                                TRAIN_DATA_DIR,
                                                x['rect'],
                                                N_SEGMENTS),
                        axis = 1)

A Jupyter Widget

1        None
6        None
7        None
8        None
9        None
10       None
12       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
27       None
28       None
29       None
31       None
32       None
33       None
34       None
35       None
36       None
38       None
39       None
40       None
         ... 
42263    None
42264    None
42267    None
42268    None
42269    None
42271    None
42277    None
42286    None
42287    None
42289    None
42290    None
42291    None
42293    None
42294    None
42296    None
42300    None
42301    None
42302    None
42304    None
42307    None
42308    None
42309    None
42310    None
42311    None
42312    None
42314    None
42316    None
42317    None
42320    None
42322    None
Length: 26129, dtype: object

In [19]:
process_image(11, TRAIN_DATA_DIR, [0, 338, 169, 450])

TypeError: process_image() missing 1 required positional argument: 'n_segments'

In [21]:
len(os.listdir('masks'))

21738